In [ ]:
import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber, client_id, client_secret, access_token, folder_id

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

import re
import matplotlib.pyplot as plt
import numpy             as np
import matplotlib.dates  as mdates
import altair            as alt

# Enable altair usage in Jupyter Notebook
alt.renderers.enable('notebook')

from   datetime          import timedelta
from   datetime          import datetime

dt = datetime.now()

import pandas            as     pd
from   pandas.plotting   import register_matplotlib_converters

register_matplotlib_converters()

from   io                import StringIO

import arable_data_dict  as dd

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

device_ids = []
device_date_created = []
for line in open(str('Report_' + str(dt_to_ymd(dt)) + '.txt'), 'r+'):
    device_ids.append(line.split(',')[1])
    device_date_created.append(line.split(',')[3])
device_ids.pop(0)   
device_ids = [[i] for i in device_ids]
device_date_created.pop(0)
#print(device_date_created)  
device_date_created_dt = [datetime.strptime(i, "%Y-%m-%d %H:%M:%S") for i in device_date_created]
#print(device_date_created_dt)

In [ ]:
import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber, client_id, client_secret, access_token, folder_id

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

start_current_month = ym_to_dt(dt_to_ym(dt)).replace(day=1)
print(start_current_month)
print(dt_to_ym(start_current_month))

today = dt

def write_csv_file(sta, end, data_type, data_type_fn, folder, cm, device): 
    logfile = 'Log_' + str(dt_to_ymd(today)) + '.txt'
    #col = ['sdi12_version', 'sdi12_vendor_id', 'sdi12_sensor_model', 'sdi12_sensor_version', 'sdi12_sensor_sn']
    fn = str(folder) + str(device)[2:-2] + '_' + str(cm) + '_' + str(data_type_fn) + '.csv'
    try:
        df = a.query(select = 'all', 
                     format = 'csv', 
                     devices = device, 
                     measure = str(data_type), 
                     order = "time", 
                     end = end, 
                     start = sta, 
                     limit = 100000000)
        df = StringIO(df)
        df = pd.read_csv(df, sep=',', error_bad_lines=False)
        df = df.dropna(axis='columns', thresh=2)#how='all')
        df = df.drop('location', 1)
        #try:
        #    for i in col:
        #        df = df.drop(i, 1)
        #except:
        #    continue
        df.to_csv(fn, sep = ",")
        #print('Successfully wrote ' + fn)
        open(logfile, 'a+').write('Successfully wrote ' + fn + '\n')
        #box_file = client.folder(folder_id).upload(str(fn), str(fn))
    except:
        open(logfile, 'a+').write('Failure writing ' + fn + '\n')
        pass  

# ---> YEAR
# --------> MONTH
# --------------> DATA TYPE
# ------------------------> DEVICE-ID_MONTH-YEAR_DATA-TYPE.csv
Folder_Name          = start_current_month.strftime("%Y")
print(Folder_Name)
Sub_Folder_Name      = start_current_month.strftime("%m")
Sub_Sub_Folder_Names = ["Daily", "Hourly", "Soil", "Calibrated"]
Data_Types           = ["daily", "hourly", "aux_raw", "health"]

Folder_Names         = [str(str(Folder_Name) + '/' + str(Sub_Folder_Name) + '/' + str(i) + '/') for i in Sub_Sub_Folder_Names]

import os

dirpath = os.getcwd()
foldername = os.path.basename(dirpath)
print(foldername)

for j in range(len(Data_Types)):
    if not os.path.exists(Folder_Names[j]):
        os.makedirs(Folder_Names[j])
        print "Directory " , Folder_Names[j] ,  " Created "
    else:    
        print "Directory " , Folder_Names[j] ,  " already exists"  
        pass
    for dev in device_ids:
        #print(dev)
        write_csv_file(start_current_month, today, Data_Types[j], Sub_Sub_Folder_Names[j], Folder_Names[j], dt_to_ym(start_current_month), dev)